# DREAM-tools

Install or upgrade dream-tools:

In [94]:
!pip install dream-tools --upgrade

Requirement already up-to-date: dream-tools in c:\users\b031441\appdata\local\continuum\miniconda3\lib\site-packages (0.3.5)


Our convention is to import the *dreamtools* package under the alias *dt*.


In [95]:
import dreamtools as dt

## Gams Pandas
DREAM-tools provides a wrapper class *GamsPandasDatabase* that enables convertion between Pandas data structures and a Gams Database.

We can either open an existing GDX file as a GamsPandasDatabase or create new database from scratch.

### Creating and populating a new GamsPandasDatabase

To create an empty database we use the GamsPandasDatabase class.

The GamsPandasDatabase has 3 main methods for creating new Gams symbols: *create_parameter*, *create_variable*, and *create_set*. Each of these methods adds a Gams symbol to the underlying Gams Database and returns a Pandas object (or a scalar) representing the symbol.

To improve readability, we can assign some shorter names (In SnakeCase to indicate that these methods essentially work as classes):

In [96]:
db = dt.GamsPandasDatabase()
Par, Var, Set = db.create_parameter, db.create_variable, db.create_set

### Define sets

In [97]:
t = Set("t", range(2010, 2020), "Årstal")
s = Set("s", ["tjenester", "fremstilling"], "Brancher")

st = Set("st", [s, t], "Branche x år dummy")
sub = Set("sub", ["tjenester"], "Subset af brancher", domains=["s"])

one2one = Set("one2one", [(2010, 2015), (2011, 2016)], "1 til 1 mapping", domains=["t", "t"])
one2many = Set("one2many",
               [("tot", "tjenester"), ("tot", "fremstilling")],
               "1 til mange mapping", domains=["*", "s"],
               )


### Define new parameters and variables



In [98]:
gq = Par("gq", None, "Produktivitets-vækst", 0.01)
fq = Par("fp", t, "Vækstkorrektionsfaktor", (1 + 0.01) ** (t - 2010))
d = Par("d", st, "Dummy")
y = Var("y", [s, t], "Produktion")
p = Var("p", [s, t], "Pris")

### Python representation of Gams symbols

A set is represented by a Pandas Index or a Pandas MultIndex if it has more than one dimension.

Scalars, such as *gq*, are simply Floats or Ints, while all other variable or parameters are accessed as Pandas Series:

In [99]:
print(type(s))
print(type(one2many))

print(type(gq))
print(type(p))

<class 'pandas.core.indexes.base.Index'>
<class 'pandas.core.indexes.multi.MultiIndex'>
<class 'float'>
<class 'pandas.core.series.Series'>


As they are regular Pandas Series, all the usual methods can be used (and googled). E.g. use reset_index to get a DataFrame from a multiindex Series:

In [100]:
p.reset_index().head()

,s,t,p
0,tjenester,2010,NaN
1,tjenester,2011,NaN
2,tjenester,2012,NaN
3,tjenester,2013,NaN
4,tjenester,2014,NaN


### Assigning to existing symbols
To change the values of the underlying Gams symbols, start by assigning values to their Pandas Series representations:

In [101]:
y["tjenester"], y["fremstilling"] = 7, 3
p[:] = 100

DREAM-tools defaults to using Pandas Series with multi-indexes rather than DataFrames.

That is because using multi-indexes greatly simplifies operations when our variables have multiple dimensions with consistent names. For example, Pandas automatically aligns the *y* and *fq* variables on their shared *t* axis without us having to do any complicated merging operations:

In [102]:
y *= fq
p * y

s             t   
tjenester     2010    700.000000
              2011    707.000000
              2012    714.070000
              2013    721.210700
              2014    728.422807
              2015    735.707035
              2016    743.064105
              2017    750.494746
              2018    757.999694
              2019    765.579691
fremstilling  2010    300.000000
              2011    303.000000
              2012    306.030000
              2013    309.090300
              2014    312.181203
              2015    315.303015
              2016    318.456045
              2017    321.640606
              2018    324.857012
              2019    328.105582
dtype: float64

Checking if a symbol name already exists in the database is as simple as

In [103]:
print("y" in db)
print("free_lunch" in db)

True
False


To figure out which type of symbol it is, we can either check which dictionary it belongs to or access the underlying Gams Symbol through the *.symbols* dictionary.

In [104]:
print("y" in db.sets)
print("y" in db.variables)
print("y" in db.parameters)

print(type(db.symbols["y"]))

False
True
False
<class 'gams.database.GamsVariable'>


Usually we only want to access the Pandas representation of a symbol. We can access those using either **\[  \]** or **.** notation.

In [105]:
print(type(db["y"]))
print(db.y is db["y"])

<class 'pandas.core.series.Series'>
True


Use **\[ \]** notation to assign a new value to an existing scalar

In [106]:
db["gq"] = 0.02

### Exporting the database to a gdx file

In [107]:
db.export("test.gdx")

## Working with an existing gdx file
If we already have a gdx file, such as the one we just exported, we can read that instead of creating a GamsPandasDatabase from scratch using the Gdx class:

In [108]:
gdx = dt.Gdx("test.gdx")
isinstance(gdx, dt.GamsPandasDatabase)

print(gdx.gq)

0.02


## Time series analysis

If the *Plotly* package is installed, DREAM-tools sets the plotting backend of Pandas to Plotly. That means Plotly is used by default when plotting or printing a Pandas objects instead of Matplotlib. Neither plotting packages handles multi-indexes well. To plot our *y* variable from befroe we could write:

In [109]:
y.unstack("s").plot()

We could also use the DREAM-tools plot function:

In [110]:
from dreamtools import plot
plot(y)

For more advanced examples, we open a gdx file as a GamsPandasDatabase and set it as the "reference database".
The reference database should be our baseline scenario for any analysis.

In [111]:
dt.REFERENCE_DATABASE = dt.Gdx("Q:/MAKRO/Data/PreviousSolutions/previous_dynamic_calibration")

We read another gdx file and assign it to a short name for easy access.

In [112]:
s = dt.Gdx("Q:/MAKRO/Data/PreviousSolutions/previous_dynamic_calibration2")

Let's try plotting a few national income variables:

In [113]:
plot([s.vX, s.vC, s.vG, s.vI])

Note that only the aggregate elements are plotted despite each variable being multidimensional.

That is because *dreamtools* is setup with a dictionary defining which elements should be shown for a number of sets if we haven't specified anything.

In [114]:
dt.DEFAULT_SET_AGGREGATIONS

{'a_': ['tot'],
 'a': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
 'c_': ['cTot'],
 'c': ['cBil', 'cEne', 'cVar', 'cBol', 'cTje', 'cTur'],
 'x_': ['xTot'],
 'x': ['xEne', 'xVar', 'xSoe', 'xTje', 'xTur'],
 'g_': ['gTot'],
 'g': ['g'],
 's_': ['tot'],
 'sp': ['tje', 'fre', 'byg', 'soe', 'bol', 'lan', 'ene', 'udv'],
 's': ['tje', 'fre', 'byg', 'soe', 'bol', 'lan', 'ene', 'udv', 'off'],
 'r_': ['rTot'],
 'r': ['tje', 'fre', 'byg', 'soe', 'bol', 'lan', 'ene', 'udv', 'off'],
 'i_': ['iTot'],
 'i': ['im', 'ib', 'iL'],
 'k_': ['im', 'ib'],
 'k': ['im', 'ib'],
 'portf': ['Net'],
 'portf_': ['Net'],
 'scen_': ['GovSpending', 'IntRate', 'ForeignDemand']}

If we want other components we can simply be more specific (or supply a different *default_set_aggregations* dictionary):

In [115]:
plot(s.vC[s.c])

To compare variables with our baseline (the reference database) instead of writing

```
s.vX / baseline.vX - 1, s.vX / baseline.vX - 1 ...
```

we can use the operator "q" (as in Gekko):

In [116]:
plot([s.vX, s.vC, s.vG, s.vI], "q", 2015, 2025)

The *prt* function works identically, except instead of returning a plotly figure for plotting, it returns a string for printing.

We can set a global time period of interest using the *time* function.

3 operators are currently implemented:
- "s": show baseline side-by-side (shock, baseline)
- "q": relative multiplier (shock / baseline - 1)
- "m": absolute multiplyer (shock - baseline)



In [122]:
from dreamtools import prt
dt.time(2017, 2025)
prt(s.vX, "s")

,vX[xTot],baseline.vX[xTot]
2017,976.289020,976.289020
2018,996.441204,996.986831
2019,989.193867,989.730600
2020,990.291596,991.494294
2021,989.704457,991.790578
2022,988.937522,992.060577
2023,987.812130,992.064379
2024,986.347819,991.763038
2025,984.671846,991.234124


Here is an example with a few more useful arguments:

In [118]:
plot([s.qY, s.qK],
        operator="m",
        start_year=2016,
        end_year=2080,
        reference_database=None,
        default_set_aggregations={"s_": ["tje"], "s": ["tje"]},
        layout={"title": "Example graph", "yaxis": {"title": "Difference from baseline"}},  # See plotly documentation for other layout options
)

### Age profiles
2 functions have been added specifically to plot age profiles more easily:

In [119]:
dt.age_figure_2d(s.vC_a, years=[2017, 2090])

In [123]:
dt.age_figure_3d(s.vC_a, end_age=99, end_year=2099)


### Exporting plots
There are many ways to export the plots, but one easy way to store and share them is to export an html file:

In [121]:
figures = [
    plot(s.vC),
    plot(s.vG)
]
dt.figures_to_html(figures, "example.html")

Or we can simply share a Jupyter notebook or export the notebook to html. To export a notebook without the code, the following command can be used:

```jupyter nbconvert examples.ipynb --to=html --TemplateExporter.exclude_input=True```